# Facility Report Analysis 2 - Garvit

In [1]:
%matplotlib inline
import re
import nltk
import gensim
import pickle
import pandas as pd
import numpy as np
import pyLDAvis.gensim
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from gensim import corpora
from sklearn.feature_extraction.text import TfidfTransformer

c:\users\garvi\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### 1. Reading all Datasets

In [2]:
with open('df_now.pkl', 'rb') as handle:
    df_now = pickle.load(handle)

In [3]:
with open('df_then.pkl', 'rb') as handle:
    df_old = pickle.load(handle)

In [4]:
with open('df_oldest.pkl', 'rb') as handle:
    df_oldest = pickle.load(handle)

In [5]:
with open('lda_docs.pkl', 'rb') as handle:
    lda_docs = pickle.load(handle)

In [6]:
with open('lda_docs_old.pkl', 'rb') as handle:
    lda_docs_old = pickle.load(handle)

In [7]:
with open('lda_docs_oldest.pkl', 'rb') as handle:
    lda_docs_oldest = pickle.load(handle)

In [5]:
df_oldest.rename(columns={'Closed On': 'Date Finished'}, inplace=True)

### 2. Reading Topics

In [6]:
topic_names = pd.read_excel('Topic_Names.xlsx')

In [7]:
topic_names.head()

,Topic #,Topic Name,Unnamed: 2,Unnamed: 3
0,1,elevator,NaN,Buidling repair - exterior
1,2,Unknown - not clear,NaN,Building repair - interior
2,3,Key - Door problems (including ISAAC System),NaN,Custodial request
3,4,Unknown - not clear,NaN,Electrical
4,5,Electrical,NaN,"Electrical - IT, WiFi, computer"


### 3. Categorizing documents based on Topics (One Dataset)

In [8]:
topic_records = {}
for i in range(1,100):
    topic = "Topic " + str(i)
    topic_name = topic_names.iloc[i-1]['Topic Name']
    if topic_name in list(topic_records):
        topic_records[topic_name] = topic_records[topic_name].union(set(list(df_now[df_now['Topics'].str.contains(topic)].index)))
    else:
        topic_records[topic_name] = set(list(df_now[df_now['Topics'].str.contains(topic)].index))

In [106]:
list(topic_records)

['elevator',
 'Unknown - not clear',
 'Key - Door problems (including ISAAC System)',
 'Electrical',
 'Renovation - room repair',
 'Pests, exterminate',
 'Plumbing - interior',
 'Maintenance - exterior (mow, weed, water)',
 'HVAC',
 'Buidling repair - exterior',
 'Electrical - IT, WiFi, computer',
 'Water treatment - pool, fountain',
 'Electrical - lighting',
 'Vehicle repair - Vehicle maintenance',
 'Building repair - interior',
 'Custodial request',
 'Fire safety',
 'Signs']

In [10]:
sum = 0 
for key,value in topic_records.items():
    print(key,len(value))
    sum += len(value)
print("SUM: ",sum)
print("DIFF:",sum-len(df_now))
print("LEN: ", len(df_now))

elevator 38309
Unknown - not clear 210732
Key - Door problems (including ISAAC System) 54993
Electrical 33982
Renovation - room repair 74376
Pests, exterminate 58456
Plumbing - interior 58087
Maintenance - exterior (mow, weed, water) 1882
HVAC 43011
Buidling repair - exterior 13784
Electrical - IT, WiFi, computer 10925
Water treatment - pool, fountain 19267
Electrical - lighting 17089
Vehicle repair - Vehicle maintenance 24300
Building repair - interior 18701
Custodial request 13033
Fire safety 6323
Signs 1517
SUM:  698767
DIFF: 430241
LEN:  268526


### 4. Aggregating all three datasets.

In [12]:
with open('df_now_HVAVC.pkl', 'wb') as handle:
    pickle.dump(df_now.iloc[list(topic_records["HVAC"])], handle)

In [78]:
df = df_old.append(df_now, ignore_index=True)

In [79]:
df = df.append(df_oldest, ignore_index=True)

In [80]:
list(df)

['Account',
 'Account Number(s)',
 'Acct No',
 'Action Requested',
 'Area',
 'Area Function',
 'Area Type',
 'Assigned Trades',
 'Building',
 'Building Number',
 'Campus',
 'Charge Code',
 'Charge Type',
 'Date Completed',
 'Date Estimate Began',
 'Date Estimate Was Completed',
 'Date Finished',
 'Date Funded',
 'Date Requested',
 'Date WO Issued',
 'Days to Completion',
 'Days to Estimate Completed',
 'Days to Finish',
 'Days to Issue',
 'Days to Start',
 'Department',
 'Dept Number',
 'Equipment Type',
 'FM Labor',
 'First Charge',
 'First Labor On',
 'First Material On',
 'Flat Bill',
 'Floor',
 'Funded Amount',
 'General Comments',
 'Job_Type',
 'Labor',
 'Labor Hours',
 'Materials',
 'NonStock On Order Not Received',
 'NonStock Received Not Invoiced',
 'NonStock Received and Invoiced',
 'Object Type',
 'Phone',
 'Priority',
 'Project',
 'Purchase Order',
 'Reference #',
 'Reference Number',
 'Req Phone',
 'Request Contact',
 'Request Source',
 'Requested By',
 'Requested On',
 'Re

In [81]:
len(df_now) + len(df_old) + len(df_oldest) 

618801

In [82]:
len(df)

618801

In [103]:
df = df.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)

In [104]:
writer = pd.ExcelWriter('Facility_Report_Dataset_All.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

C:\Program Files\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:830: UserWarning:

Ignoring URL 'https://www.dropbox.com/sc/bsernfjsy338hnn/X7BO-rY-Eb%20%20%20%20Please%20see%20the%20attached%20photos%20of%20a%20broken%20rail%20on%20the%20loading%20dock%20of%20Hayden%20Library.%20%20%20The%20rail%20is%20loose%20and%20is%20a%20safety%20hazard.%20%20%20I%20don't%20have%20any%20information%20on%20how%20the%20rail%20was%20broken.%20%20%20%20The%20pictures%20were%20not%20attached%20%20%20%2042B1L1O' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS



In [105]:
import pickle
with open('df_all.pkl', 'wb') as handle:
    pickle.dump(df, handle)

In [83]:
len(df)

618801

### 5. Extracting Bulding Number to each records for LEED Building Analysis.

In [14]:
BldgNo = []
for bldg in list(df_now['Building'].str.split(' ')):
    if type(bldg) is list:
        BldgNo.append(bldg[0].strip())
    else:
        BldgNo.append("")

In [108]:
BldgNo

['024B',
 '057A',
 '024A',
 '084',
 '046',
 '136',
 '069',
 '130',
 '053C',
 '040',
 '131',
 '127',
 '077X',
 '027',
 '010',
 '056E',
 '045',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '059',
 '136',
 '077X',
 '046',
 '049',
 '049',
 '077X',
 '084',
 '017',
 '010',
 '039',
 '049',
 '057A',
 '077X',
 '127',
 '013',
 '044',
 '150',
 '065',
 '174',
 '035',
 '130',
 '039',
 '004',
 '004',
 '036',
 '056A',
 '069',
 '077X',
 '059B',
 '012',
 '003',
 '010',
 '030',
 '057B',
 '085',
 '004',
 '077X',
 '077X',
 '077X',
 '077X',
 '515',
 '166',
 '165B',
 '156',
 '150',
 '150',
 '150',
 '130',
 '104A',
 '095A',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '077X',
 '0

In [16]:
df_now['Building Number'] = BldgNo

In [17]:
BldgNo = []
for bldg in list(df_old['Building'].str.split(' ')):
    if type(bldg) is list:
        BldgNo.append(bldg[0].strip())
    else:
        BldgNo.append("")

In [18]:
df_old['Building Number'] = BldgNo

In [19]:
BldgNo = []
for bldg in list(df_oldest['Building'].str.split(' ')):
    if type(bldg) is list:
        BldgNo.append(bldg[0].strip())
    else:
        BldgNo.append("")

In [20]:
df_oldest['Building Number'] = BldgNo

### 5. Classifying LEED Buildings.

In [75]:
leed_df = pd.read_excel('ASU_LEED_Bldgs.xlsx')
leed_data = pd.read_excel('ASU_LEED_Bldgs.xlsx')

In [76]:
list(leed_df)

['BldgNo',
 'Status',
 'LEED Certification Date',
 'LEED Certification Year',
 'Project',
 'LEED Certification Level',
 'BldgUse',
 'Campus',
 'Building Name',
 'Sq.Ft.',
 'KWH Energy Use']

In [23]:
leed_df = leed_df[leed_df['LEED Certification Year']>=2011]

In [77]:
leed_df['BldgNo']

0       054
1      054A
2      E816
3      104B
4      104A
5       R11
6       108
7       035
8     E827B
9     E825A
10    E825B
11    E827A
12     E826
13      007
14      305
15     160E
16     163C
17     160B
18     160D
19     160C
20     163A
21     160A
22     163B
23     3001
24      R50
25      168
26      307
27     173B
28     173C
29     173D
30     173H
31     173E
32     173A
33     173F
34     173G
35      174
36      075
37      914
38     040D
39      915
40     E831
41      025
42     E830
43      061
44      178
45      308
46      046
47      ???
48      309
49     104C
50      917
51      ???
52      059
Name: BldgNo, dtype: object

In [84]:
leed_buildings = list(leed_df['BldgNo'])

In [85]:
Leed_Certification = []
for row in range(len(df)):
    if df.iloc[row]['Building Number'] in leed_buildings:
        Leed_Certification.append(True)
    else:
        Leed_Certification.append(False)

In [86]:
len(Leed_Certification)

618801

In [87]:
Leed_Certification

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [88]:
df[df['Building Number'].str.contains("061")]

,Account,Account Number(s),Acct No,Action Requested,Area,Area Function,Area Type,Assigned Trades,Building,Building Number,...,Tag Number,Task Comments,Topics,Total,Trades,WO Object,WO Type,Warehouse,Warehouse Parts,Work Order
12315,NaN,NaN,NO CHRG,Manzanita Pump Pit: The condensate line is le...,NaN,NaN,NaN,"AC, PL",061 (CB) - MANZANITA HALL,061,...,NaN,2015-06-16 - 26071 - Cheryl stated that Wayne ...,"Topic 68,Topic 35,Topic 12,Topic 44,Topic 99,T...",5449.62,NaN,061 (CB) - MANZANITA HALL,Corrective Maintenance,2579.82,NaN,TMPE-2015054235
12559,NaN,NaN,GN71165E2,Manzanita: ASU HD Robert from AC shop resp...,NaN,NaN,NaN,AC,061 (CB) - MANZANITA HALL,061,...,NaN,2015-06-19 - 20000 - Robert Dauscher-Contacted...,"Topic 87,Topic 32,Topic 34,Topic 84,Topic 83,T...",57.02,NaN,061 (CB) - MANZANITA HALL,Service,0.00,NaN,TMPE-2015053991
14244,NaN,NaN,NO CHRG,Manzanitia: ASU HD: 05-30-2015 14:02:10 MST ...,NaN,NaN,NaN,EL,061 (CB) - MANZANITA HALL,061,...,NaN,2015-06-01 - 16000 - Work order was issued as ...,"Topic 93,Topic 32,Topic 57,Topic 90,Topic 55,T...",114.03,NaN,061 (CB) - MANZANITA HALL,Corrective Maintenance,0.00,NaN,TMPE-2015052306
15426,NaN,NaN,GN71165E2,Manzanita Pump Pit: Add AC shop: Bill The c...,NaN,NaN,NaN,AC,061 (CB) - MANZANITA HALL,061,...,NaN,2015-05-21 - 26071 - William Zimmer-Layout and...,"Topic 71,Topic 12,Topic 8,Topic 38,Topic 93,To...",1836.87,NaN,061 (CB) - MANZANITA HALL,Service,0.00,NaN,TMPE-2015051124
15468,NaN,NaN,JC51038-JC502001074-01,MANZANITA HALL - #2 Entrapment,NaN,NaN,NaN,OF,061 (CB) - MANZANITA HALL,061,...,NaN,NaN,"Topic 44,Topic 81,Topic 84",0.00,NaN,061 (CB) - MANZANITA HALL,Service,0.00,NaN,TMPE-2015051082
16621,NaN,NaN,NO CHRG,BIKE LOCK CUT - front bike rack at Manzanita H...,NaN,NaN,NaN,AC,061 (CB) - MANZANITA HALL,061,...,NaN,2015-05-11 - 20090 - Mitchell Andrews-cut lock...,"Topic 98,Topic 46,Topic 3,Topic 9,Topic 63,Top...",38.01,NaN,061 (CB) - MANZANITA HALL,Corrective Maintenance,0.00,NaN,TMPE-2015049929
18079,NaN,NaN,ET11013-ET102000058-01,Manzanita Hall: SCOPE OF WORK – MECHANICAL IN...,NaN,NaN,NaN,TS,061 (CB) - MANZANITA HALL,061,...,NaN,NaN,"Topic 9,Topic 78,Topic 81,Topic 59,Topic 57,To...",0.00,NaN,061 (CB) - MANZANITA HALL,Service,0.00,NaN,TMPE-2015048471
18676,NaN,NaN,NM21002,Manzanita - 0859: Freight ELV is down,061 (CB)-1000E4 - Circulation Area,NaN,WWW - Circulation Area,OF,061 (CB) - MANZANITA HALL,061,...,NaN,2015-04-23 - 17005 - 2183118 - Thyssen - Lenwo...,"Topic 31,Topic 36,Topic 38,Topic 3,Topic 79,To...",0.00,NaN,1000E4 - Circulation Area,Service,0.00,NaN,TMPE-2015047874
19805,NaN,NaN,GN7116586,Manzy - Dining Hall: Back Dock The back ...,NaN,NaN,NaN,KY,061 (CB) - MANZANITA HALL,061,...,NaN,2015-05-04 - 11000 - John Ritzko-repaired lock...,"Topic 66,Topic 21",44.53,NaN,061 (CB) - MANZANITA HALL,Service,0.00,NaN,TMPE-2015046745
22754,NaN,NaN,GN71165E2,Manzanita Pump Pit: The condensate line is le...,NaN,NaN,NaN,"AC, PL",061 (CB) - MANZANITA HALL,061,...,NaN,2015-03-25 - 26071 - Cheryl stated that Wayne ...,"Topic 68,Topic 81,Topic 38,Topic 78,Topic 12,T...",0.00,NaN,061 (CB) - MANZANITA HALL,Service,0.00,NaN,TMPE-2015043796


In [89]:
df['Leed Certification'] = Leed_Certification

In [90]:
len(df[df['Leed Certification']==True])

118603

In [93]:
leed_df = df[df['Leed Certification'] == True]

In [94]:
leed_df['Building'].unique()

array(['915 - SUN DEVIL FITNESS COMPLEX - 915',
       '914 - VERDE DINING PAVILION', '104A - BIODESIGN INSTITUTE BLDG A',
       '104B (CM) - BIODESIGN INSTITUTE BLDG B',
       '025 (CS) - HEALTH SERVICE', '178 (CS) - COLLEGE AVENUE COMMONS',
       '007 - WRIGLEY HALL', '168 - ASU POLICE',
       '035 (CB) - MEMORIAL UNION',
       '108 (CM) - INTERDISCIPLINARY SCIENCE AND TECHNOLOGY BUILDING I',
       '040D - MCCORD HALL', '061 (CB) - MANZANITA HALL',
       '046 (CS) - SUN DEVIL FITNESS COMPLEX - 046',
       '075 (CM) - INTERDISCIPLINARY SCIENCE AND TECHNOLOGY BUILDING 4',
       'R11 (CB) - FULTON CENTER',
       '054 (CM) - INTERDISCIPLINARY SCIENCE AND TECHNOLOGY BUILDING II',
       '173H (CB) - BARRETT COLLEGE HONORS HALL',
       '059 (CB) - SUN DEVIL STADIUM', '174 (CB) - WEATHERUP CENTER',
       '173A (CB) - BARRETT COLLEGE SAGE HALL', '305 - CRONKITE/8',
       '163B (CB) - HAV- VERBENA HALL',
       '173D (CB) - BARRETT COLLEGE COTTONWOOD HALL',
       '163A (CB) - HA

### 6. Categorizing documents based on Topics (All three Dataset)

In [91]:
df = df[df['Date Finished']!='- -']
df = df[df['Total']!=0]

In [92]:
len(df)

237199

In [95]:
topic_records_all = {}
for i in range(1,101):
    topic = "Topic " + str(i)
    topic_name = topic_names.iloc[i-1]['Topic Name']
    if topic_name in list(topic_records):
        topic_records_all[topic_name] = topic_records[topic_name].union(set(list(leed_df[leed_df['Topics'].str.contains(topic)].index)))
    else:
        topic_records_all[topic_name] = set(list(leed_df[leed_df['Topics'].str.contains(topic)].index))

In [96]:
for key,value in topic_records_all.items():
    print(key,len(value))

elevator 53669
Unknown - not clear 211082
Key - Door problems (including ISAAC System) 57073
Electrical 34781
Renovation - room repair 75546
Pests, exterminate 62597
Plumbing - interior 58502
Maintenance - exterior (mow, weed, water) 6910
HVAC 44672
Buidling repair - exterior 15646
Electrical - IT, WiFi, computer 14670
Water treatment - pool, fountain 21027
Electrical - lighting 21148
Vehicle repair - Vehicle maintenance 24670
Building repair - interior 22656
Custodial request 15565
Fire safety 7867
Signs 4655


In [97]:
new_df = leed_df

In [98]:
len(new_df)

38765

### 7. Number of Issues vs Buildings (Check whether we have sufficient data)

In [11]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)

In [99]:
buildings = new_df['Building'].unique()

data = []
for building in buildings:
    building_df = new_df[new_df['Building']==building]
    try:
        building_df = building_df.groupby(building_df['Date Finished'].dt.strftime('%Y-%m'))['Total'].count()
    except:
        print(building_df)
    
    data.append(Scatter(
        x = building_df.index,
        y = list(building_df),
        mode = 'line',
        name = building
    ))

layout = Layout(xaxis = dict(title = 'Date Finished'), yaxis = dict(title = 'Number of Issues'), 
                title =  'LEED Buildings: Date Finished v/s Number of Issues (per Building)')
# data = [trace0]

fig = {'data' : data, 'layout' : layout}

iplot(fig)
plot(fig,  filename='LEED_Frequency_All.html')

'file://C:\\Users\\garvi\\PycharmProjects\\NEPTUNE_Facility_Reports\\LEED_Frequency_All.html'

In [100]:
topic = "HVAC"
new_df = leed_df.loc[list(topic_records_all[topic])]

### 8. Leed Building vs Cost (Dotted line shows the Leed Certification Date)

In [112]:
data = []
bno = "025"
maxVal = 0

certification_date = leed_data[leed_data['BldgNo'] == bno]["LEED Certification Date"]
certification_date = certification_date.dt.strftime('%Y-%m')

for topic in list(topic_records_all):
    topic_df = leed_df.loc[list(topic_records_all[topic])]    
    building = topic_df[topic_df['Building Number']==bno]["Building"].iloc[0]
    topic_df = topic_df[topic_df['Building Number']==bno]
    
    try:
        topic_df = topic_df.groupby(topic_df['Date Finished'].dt.strftime('%Y-%m'))['Total'].sum()
    except:
        print(topic_df)
    
    data.append(Scatter(
        x = topic_df.index,
        y = list(topic_df),
        mode = 'line',
        name = topic
    ))
    
    if np.max(topic_df) > maxVal:
        maxVal = np.max(topic_df)

temp_df = pd.DataFrame(index=certification_date, 
                  data=[0])

temp_df.index.rename('Date Finished', inplace=True)
temp_df = topic_df.append(temp_df, ignore_index = False)

data.append(Scatter(x=[temp_df.index[-1], temp_df.index[-1]], y=[0, maxVal],
        line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4,
        dash = 'dash'),
        name = "Leed Certification Date"))

layout = Layout(xaxis = dict(title = 'Date Finished'), yaxis = dict(title = 'Total Cost'), 
                title = building + ': Date Finished v/s Total Cost (per Topic)')
# data = [trace0]

fig = {'data' : data, 'layout' : layout}

iplot(fig)
plot(fig,  filename="Topic_" + str(building) + '_cost.html')

'file://C:\\Users\\garvi\\PycharmProjects\\NEPTUNE_Facility_Reports\\Topic_025 (CS) - HEALTH SERVICE_cost.html'

### 9. Leed Building vs Frequency (Dotted line shows the Leed Certification Date)

In [119]:
data = []
bno = "025"
maxVal = 0

certification_date = leed_data[leed_data['BldgNo'] == bno]["LEED Certification Date"]
certification_date = certification_date.dt.strftime('%Y-%m')

for topic in list(topic_records_all):
    topic_df = leed_df.loc[list(topic_records_all[topic])]    
    building = topic_df[topic_df['Building Number']==bno]["Building"].iloc[0]
    topic_df = topic_df[topic_df['Building Number']==bno]
    
    try:
        topic_df = topic_df.groupby(topic_df['Date Finished'].dt.strftime('%Y-%m'))['Total'].count()
    except:
        print(topic_df)
    
    data.append(Scatter(
        x = topic_df.index,
        y = list(topic_df),
        mode = 'line',
        name = topic
    ))
    
    if np.max(topic_df) > maxVal:
        maxVal = np.max(topic_df)

temp_df = pd.DataFrame(index=certification_date, 
                  data=[0])

temp_df.index.rename('Date Finished', inplace=True)
temp_df = topic_df.append(temp_df, ignore_index = False)

data.append(Scatter(x=[temp_df.index[-1], temp_df.index[-1]], y=[0, maxVal],
        line = dict(
        color = ('rgb(205, 12, 24)'),
        width = 4,
        dash = 'dash'),
        name = "Leed Certification Date"))

layout = Layout(xaxis = dict(title = 'Date Finished'), yaxis = dict(title = 'Frequency'), 
                title = building + ': Date Finished v/s Frequency (per Topic)')
# data = [trace0]

fig = {'data' : data, 'layout' : layout}

iplot(fig)
plot(fig,  filename="Topic_" + str(building) + '_cost.html')

'file://C:\\Users\\garvi\\PycharmProjects\\NEPTUNE_Facility_Reports\\Topic_025 (CS) - HEALTH SERVICE_cost.html'

### 10. Few Experiments for Future
a. Use "Date Requested" instead of "Date Finished". <br/>
b. Increase the threshold for topic classification, which decides what topic does a document belongs to. Currently a lot of documents are getting tagged to several topics.<br/>
c. Normalize the data consider area of the buildings, area of campuses, number of buildings, etc.<br/>
d. Reliability Prediction using Weibull Analysis.<br/>
e. Find better way to classify "AS400 WOs.xlsx".<br/>